In [1]:
import numpy as np
import pandas as pd

In [2]:
import helpers

In [3]:
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io

plotly.io.renderers.default = "jupyterlab+png"

In [4]:
helpers.datasets.load_tcga_skcm()

sample_id,TCGA-3N-A9WB-06A-11R-A38C-07,TCGA-3N-A9WC-06A-11R-A38C-07,TCGA-3N-A9WD-06A-11R-A38C-07,TCGA-BF-A1PU-01A-11R-A18S-07,TCGA-BF-A1PV-01A-11R-A18U-07,TCGA-BF-A1PX-01A-12R-A18T-07,TCGA-BF-A1PZ-01A-11R-A18S-07,TCGA-BF-A1Q0-01A-21R-A18S-07,TCGA-BF-A3DJ-01A-11R-A20F-07,TCGA-BF-A3DL-01A-11R-A20F-07,...,TCGA-XV-AB01-06A-12R-A40A-07,TCGA-YD-A89C-06A-11R-A37K-07,TCGA-YD-A9TA-06A-11R-A39D-07,TCGA-YD-A9TB-06A-12R-A40A-07,TCGA-YG-AA3N-01A-11R-A38C-07,TCGA-YG-AA3O-06A-11R-A38C-07,TCGA-YG-AA3P-06A-11R-A38C-07,TCGA-Z2-A8RT-06A-11R-A37K-07,TCGA-Z2-AA3S-06A-11R-A39D-07,TCGA-Z2-AA3V-06A-11R-A39D-07
gene_symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,381.0662,195.1822,360.8794,176.3994,216.8470,285.0608,424.5848,327.8758,232.2936,380.7694,...,428.9256,184.5297,88.4289,264.5184,238.8846,207.9831,33.5642,336.4070,349.4478,717.2209
A1CF,0.0000,0.0000,0.7092,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,1.3482,0.0000,0.0000,0.0000,0.0000,0.3876,0.0000,0.0000
A2BP1,0.0000,0.0000,6.3830,1.2987,0.0000,0.0000,0.0000,0.0000,0.0000,0.4901,...,0.0000,0.0000,0.0000,0.0000,0.5522,0.3380,66.0629,0.0000,0.2880,0.6539
A2LD1,250.1979,160.7548,97.1986,163.2338,60.8727,125.6900,78.8939,67.7049,92.9554,99.8579,...,47.5207,102.4404,36.9476,146.3544,88.2441,86.2831,50.0426,37.3178,152.9786,50.7438
A2M,2209.5194,169236.6356,18257.8865,6716.4513,1740.5828,32098.0014,3218.6418,6468.7807,377755.5402,7726.0377,...,17450.3581,20639.7659,12397.0606,25477.7594,10300.9056,37319.7533,8217.0591,22129.9496,4040.2938,46111.4664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,1236.2682,3594.4996,3270.2128,5132.1429,5128.2128,4675.7039,3177.7557,10974.0171,3042.2195,7198.7258,...,11526.1708,5296.0555,3701.5035,3021.2879,10181.1154,3125.3063,1925.4129,2865.1163,2136.9429,11832.5977
ZZEF1,622.7787,883.6696,1105.6738,1796.1039,1097.4298,1398.3316,1648.7663,1287.7062,1335.5161,1472.6783,...,1464.1873,852.6225,1245.0696,1078.7653,2362.7830,2671.3984,1480.5541,1849.6124,1640.4349,1413.1110
ZZZ3,721.3247,1137.8099,769.5035,286.0390,1658.9958,521.0288,651.4399,858.2361,962.2609,300.9066,...,224.5179,743.3897,958.9125,741.8840,1066.8139,940.5999,1003.7294,454.2636,1452.3724,102.0108


In [ ]:
mixtures_tcga_skcm = helpers.datasets.load_tcga_skcm()

In [ ]:
from sklearn.decomposition import PCA

# PCA of expression by gene

## without standardizing

In [ ]:
df = mixtures_tcga_skcm.copy().T

In [ ]:
print(df.iloc[:, :10].mean())
print(df.iloc[:, :10].std())

In [ ]:
features = list(df.columns)
pca = PCA()
components = pca.fit_transform(df)

In [ ]:
labels = {
    str(i): f"PC {i+1}<br>({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

In [ ]:
components.shape

In [ ]:
pca.explained_variance_[:10]

In [ ]:
sum(pca.explained_variance_)

In [ ]:
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(6),
    hover_name=df.index,
)
fig.update_traces(diagonal_visible=False)
fig.show(width=800, height=800)

## with standardizing

In [ ]:
df = mixtures_tcga_skcm.copy().T

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df)
df.values[:] = scaler.transform(df)

In [ ]:
print(df.iloc[:, :10].mean())
print(df.iloc[:, :10].std())

In [ ]:
features = list(df.columns)
pca = PCA()
components = pca.fit_transform(df)

In [ ]:
pca.explained_variance_[:10]

In [ ]:
sum(pca.explained_variance_)

In [ ]:
labels = {
    str(i): f"PC {i+1}<br>({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

In [ ]:
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(6),
    hover_name=df.index,
)
fig.update_traces(diagonal_visible=False)
fig.show(width=800, height=800)

In [ ]:
components.shape

In [ ]:
pca.components_.shape, len(df.columns), len(labels)

In [ ]:
u = pd.DataFrame(pca.components_.T, index=df.columns, columns=labels.values())

In [ ]:
(u**2).sum()

In [ ]:
u.apply(np.linalg.norm)

In [ ]:
u.T @ u

In [ ]:
# is this basically the identity matrix?

np.allclose(u.T @ u, np.eye(len(u.T)))

In [ ]:
u

In [ ]:
eigenvectors = pd.DataFrame(
    pca.components_.T, index=df.columns, columns=labels.values()
)

In [ ]:
eigenvectors

In [ ]:
explained_variance = pd.DataFrame(pca.explained_variance_, index=labels.values())

In [ ]:
explained_variance

In [ ]:
loadings = eigenvectors * np.sqrt(pca.explained_variance_)

In [ ]:
loadings.apply(np.linalg.norm)